<a href="https://colab.research.google.com/github/macgyver121/DADS7202_final_project/blob/main/prototypeV3_evaluate%2Bmd_k2t_25_12_65_4000d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Check GPU

In [1]:
# List all NVIDIA GPUs as avaialble in this computer (or Colab's session)
!nvidia-smi -L

GPU 0: A100-SXM4-40GB (UUID: GPU-5abd0e31-e329-1362-4bdc-9572796cb023)


In [2]:
import sys
print( f"Python {sys.version}\n" )

import numpy as np
print( f"NumPy {np.__version__}\n" )

import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
print( f"TensorFlow {tf.__version__}" )
print( f"tf.keras.backend.image_data_format() = {tf.keras.backend.image_data_format()}" )

# Count the number of GPUs as detected by tensorflow
gpus = tf.config.list_physical_devices('GPU')
print( f"TensorFlow detected { len(gpus) } GPU(s):" )
for i, gpu in enumerate(gpus):
  print( f".... GPU No. {i}: Name = {gpu.name} , Type = {gpu.device_type}" )

Python 3.8.16 (default, Dec  7 2022, 01:12:13) 
[GCC 7.5.0]

NumPy 1.21.6

TensorFlow 2.9.2
tf.keras.backend.image_data_format() = channels_last
TensorFlow detected 1 GPU(s):
.... GPU No. 0: Name = /physical_device:GPU:0 , Type = GPU


Install pytesseract

In [3]:
!sudo apt install tesseract-ocr
!pip install pytesseract
!pip install -q transformers datasets
!pip install -q pytorch-lightning wandb

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 20 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 2s (2,059 kB/s)
debconf: unable to initi

     |████████████████████████████████| 5.8 MB 30.5 MB/s 
     |████████████████████████████████| 452 kB 94.9 MB/s 
     |████████████████████████████████| 182 kB 96.9 MB/s 
     |████████████████████████████████| 7.6 MB 84.7 MB/s 
     |████████████████████████████████| 132 kB 75.3 MB/s 
     |████████████████████████████████| 212 kB 79.9 MB/s 
     |████████████████████████████████| 127 kB 76.4 MB/s 
     |████████████████████████████████| 800 kB 30.5 MB/s 
     |████████████████████████████████| 1.9 MB 69.6 MB/s 
     |████████████████████████████████| 512 kB 87.6 MB/s 
     |████████████████████████████████| 125 kB 95.1 MB/s 
     |████████████████████████████████| 182 kB 96.6 MB/s 
     |████████████████████████████████| 174 kB 83.9 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
     |████████████████████████████████| 173 kB 93.7 MB/s 
     |████████████████████████████████| 168 kB 100.6 MB/s 
     |████████████████████████████████| 168 kB 94.9 MB/s 
     |█████████

Reset runtime

In [ ]:
import os
os.kill(os.getpid(), 9)

Import library

In [1]:
import cv2
import pytesseract
import shutil
import os
import random
from PIL import Image

import re
import csv
import os

import pyarrow as pa
from datasets import DatasetDict
from datasets.arrow_dataset import Dataset

Mount drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import re
import csv
import os
import pyarrow as pa
import torch
import pyarrow as pa
from datasets import DatasetDict
from datasets.arrow_dataset import Dataset

## Create Data from Chart2text data

In [80]:
List_key = []
# Open the CSV file for reading
for i in range(4000): 
  with open('/content/drive/MyDrive/DADS7202_final/Chart2text/data/'+str(i)+'.csv', 'r') as d:
 # Read the contents of the file into a list of rows
    rows = list(csv.reader(d))
# Concatenate the rows into a single string
    csv_string = '\n'.join(['\t'.join(row) for row in rows])
    csv_string = re.sub('\s+',' ',csv_string)
    List_key.append(csv_string)
len(List_key)

4000

In [81]:
import pickle

with open('List_key_0to4000.pkl', 'wb') as f:
    pickle.dump(List_key, f)

Load

In [5]:
import pickle

with open('List_key_0to4000.pkl', 'rb') as f:
    List_key = pickle.load(f)

In [6]:
len(List_key)

4000

In [7]:
txt_key = []
for i in range(4000): 
 f = open('/content/drive/MyDrive/DADS7202_final/Chart2text/captions/'+str(i)+'.txt', 'r')
 content = f.read()
 content = re.sub('\s+',' ',content)
 txt_key.append(content)
len(txt_key)

4000

In [8]:
import pickle

with open('txt_key_0to4000.pkl', 'wb') as f:
    pickle.dump(txt_key, f)

Load

In [9]:
import pickle

with open('txt_key_0to4000.pkl', 'rb') as f:
    txt_key = pickle.load(f)

In [10]:
print(List_key[0])

Quarter Number of users in millions Q4 '19 2498 Q3 '19 2449 Q2 '19 2414 Q1 '19 2375 Q4 '18 2320 Q3 '18 2271 Q2 '18 2234 Q1 '18 2196 Q4 '17 2129 Q3 '17 2072 Q2 '17 2006 Q1 '17 1936 Q4 '16 1860 Q3 '16 1788 Q2 '16 1712 Q1 '16 1654 Q4 '15 1591 Q3 '15 1545 Q2 '15 1490 Q1 '15 1441 Q4 '14 1393 Q3 '14 1350 Q2 '14 1317 Q1 '14 1276 Q4 '13 1228 Q3 '13 1189 Q2 '13 1155 Q1 '13 1110 Q4 '12 1056 Q3 '12 1007 Q2 '12 955 Q1 '12 901 Q4 '11 845 Q3 '11 800 Q2 '11 739 Q1 '11 680 Q4 '10 608 Q3 '10 550 Q2 '10 482 Q1 '10 431 Q4 '09 360 Q3 '09 305 Q2 '09 242 Q1 '09 197 Q3 '08 100


In [11]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
# Create a list of lists that represents the data
train_data = [List_key[:2800] ,txt_key[:2800]]

# Create a list of pyarrow.Array objects from the data
arrays_train = [pa.array(col) for col in train_data]

# Create a pyarrow Table object from the arrays
table1 = pa.Table.from_arrays(arrays_train, names=['key', 'text'])
#print(type(table1))

# Create a Dataset object from the table
dataset1 = Dataset(table1)
#print(type(dataset1))

# Create a list of lists that represents the data
val_data = [List_key[2800:3400] ,txt_key[2800:3400]]

# Create a list of pyarrow.Array objects from the data
arrays_val = [pa.array(col) for col in val_data]

# Create a pyarrow Table object from the arrays
table2 = pa.Table.from_arrays(arrays_val, names=['key', 'text'])
#print(type(table1))

# Create a Dataset object from the table
dataset2 = Dataset(table2)
print(type(dataset2))

# Create a list of lists that represents the data
test_data = [List_key[3400:] ,txt_key[3400:]]

# Create a list of pyarrow.Array objects from the data
arrays_test = [pa.array(col) for col in test_data]

# Create a pyarrow Table object from the arrays
table3 = pa.Table.from_arrays(arrays_test, names=['key', 'text'])
#print(type(table1))

# Create a Dataset object from the table
dataset3 = Dataset(table3)
print(type(dataset3))

# Create a dictionary that maps keys to the Dataset objects
dataset = {
    'train': dataset1,
    'validation': dataset2,
    'test': dataset3
}

# Access a Dataset object using its key
print(dataset)  # Output: Dataset object with train_data
print(type(dataset))
#print(dataset['validation'])  # Output: Dataset object with validation_data
#print(dataset['test'])  # Output: Dataset object with test_data

<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>
{'train': Dataset({
    features: ['key', 'text'],
    num_rows: 2800
}), 'validation': Dataset({
    features: ['key', 'text'],
    num_rows: 600
}), 'test': Dataset({
    features: ['key', 'text'],
    num_rows: 600
})}
<class 'dict'>


In [13]:
from datasets import DatasetDict

dataset_dict = DatasetDict(dataset)
print(type(dataset_dict))
print(dataset_dict)

<class 'datasets.dataset_dict.DatasetDict'>
DatasetDict({
    train: Dataset({
        features: ['key', 'text'],
        num_rows: 2800
    })
    validation: Dataset({
        features: ['key', 'text'],
        num_rows: 600
    })
    test: Dataset({
        features: ['key', 'text'],
        num_rows: 600
    })
})


In [14]:
print(dataset['train'][1])

{'key': "NBA player Number of points scored Kareem Abdul-Jabbar 38387 Karl Malone 36928 LeBron James* 33655 Kobe Bryant 33643 Michael Jordan 32292 Dirk Nowitzki 31560 Wilt Chamberlain 31419 Shaquille O'Neal 28596 Moses Malone 27409 Elvin Hayes 27313 Hakeem Olajuwon 26946 Oscar Robertson 26710 Dominique Wilkins 26668 Tim Duncan 26496 Paul Pierce 26397 John Havlicek 26395 Kevin Garnett 26071 Carmelo Anthony* 26067 Vince Carter* 25623 Alex English 25613 Reggie Miller 25279 Jerry West 25192 Patrick Ewing 24815 Ray Allen 24505 Allen Iverson 24368 Charles Barkley 23757 Robert Parish 23334 Adrian Dantley 23177 Dwyane Wade 23165 Elgin Baylor 23149 Kevin Durant* 22940", 'text': "How many points does LeBron James have ? In January 2020 , LeBron James passed Kobe Bryant on the NBA all-time scoring list , scoring 29 points to in a regular season game against the Philadelphia 76ers . If James continues to produce at about his career average , he has a shot to move up even further and pass Karl Malo

In [15]:
example = dataset_dict['train'][1]

print("Key:", example["key"])
print("Text:", example["text"])

Key: NBA player Number of points scored Kareem Abdul-Jabbar 38387 Karl Malone 36928 LeBron James* 33655 Kobe Bryant 33643 Michael Jordan 32292 Dirk Nowitzki 31560 Wilt Chamberlain 31419 Shaquille O'Neal 28596 Moses Malone 27409 Elvin Hayes 27313 Hakeem Olajuwon 26946 Oscar Robertson 26710 Dominique Wilkins 26668 Tim Duncan 26496 Paul Pierce 26397 John Havlicek 26395 Kevin Garnett 26071 Carmelo Anthony* 26067 Vince Carter* 25623 Alex English 25613 Reggie Miller 25279 Jerry West 25192 Patrick Ewing 24815 Ray Allen 24505 Allen Iverson 24368 Charles Barkley 23757 Robert Parish 23334 Adrian Dantley 23177 Dwyane Wade 23165 Elgin Baylor 23149 Kevin Durant* 22940
Text: How many points does LeBron James have ? In January 2020 , LeBron James passed Kobe Bryant on the NBA all-time scoring list , scoring 29 points to in a regular season game against the Philadelphia 76ers . If James continues to produce at about his career average , he has a shot to move up even further and pass Karl Malone with a

In [16]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForSeq2SeqLM

*************** เปลี่ยน tokenizer เป็นตาม huggingface เลือก model ****************

In [18]:
tokenizer = AutoTokenizer.from_pretrained("t5-base")  ## edit here

prefix = "Text generation from keyword: "
max_input_length = 512
max_target_length = 256

def preprocess_examples(examples):
  # encode the code-docstring pairs
  codes = examples['key']
  docstrings = examples['text']
  
  inputs = [prefix + code for code in codes]
  model_inputs = tokenizer(inputs, max_length=max_input_length, padding="max_length", truncation=True)

  # encode the summaries
  labels = tokenizer(docstrings, max_length=max_target_length, padding="max_length", truncation=True).input_ids

  # important: we need to replace the index of the padding tokens by -100
  # such that they are not taken into account by the CrossEntropyLoss
  labels_with_ignore_index = []
  for labels_example in labels:
    labels_example = [label if label != 0 else -100 for label in labels_example]
    labels_with_ignore_index.append(labels_example)
  
  model_inputs["labels"] = labels_with_ignore_index

  return model_inputs

Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [19]:
dataset = dataset_dict.map(preprocess_examples, batched=True)

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [20]:
from torch.utils.data import DataLoader

dataset.set_format(type="torch", columns=['input_ids', 'attention_mask', 'labels'])
train_dataloader = DataLoader(dataset['train'], shuffle=True, batch_size=8)
valid_dataloader = DataLoader(dataset['validation'], batch_size=4)
test_dataloader = DataLoader(dataset['test'], batch_size=4)

In [21]:
batch = next(iter(train_dataloader))
print(batch.keys())

dict_keys(['input_ids', 'attention_mask', 'labels'])


In [22]:
!pip install -q pytorch-lightning wandb

******* ปรับ epoch กับ model เป็นตาม huggingface **********

### Create CodeT5 (model1)

In [28]:
from transformers import T5ForConditionalGeneration, AdamW, get_linear_schedule_with_warmup
import pytorch_lightning as pl

class CodeT5(pl.LightningModule):
    def __init__(self, lr=5e-5, num_train_epochs=100, warmup_steps=100):  ## edit here
        super().__init__()
        self.model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")  ## edit here
        self.save_hyperparameters()

    def forward(self, input_ids, attention_mask, labels=None):     
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        return outputs
    
    def common_step(self, batch, batch_idx):
        outputs = self(**batch)
        loss = outputs.loss

        return loss
      
    def training_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     
        # logs metrics for each training_step,
        # and the average across the epoch
        self.log("training_loss", loss)

        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     
        self.log("validation_loss", loss, on_epoch=True)

        return loss

    def test_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     

        return loss

    def configure_optimizers(self):
        # create optimizer
        optimizer = AdamW(self.parameters(), lr=self.hparams.lr)
        # create learning rate scheduler
        num_train_optimization_steps = self.hparams.num_train_epochs * len(train_dataloader)
        lr_scheduler = {'scheduler': get_linear_schedule_with_warmup(optimizer,
                                                    num_warmup_steps=self.hparams.warmup_steps,
                                                    num_training_steps=num_train_optimization_steps),
                        'name': 'learning_rate',
                        'interval':'step',
                        'frequency': 1}
        
        return {"optimizer": optimizer, "lr_scheduler": lr_scheduler}

    def train_dataloader(self):
        return train_dataloader

    def val_dataloader(self):
        return valid_dataloader

    def test_dataloader(self):
        return test_dataloader

In [29]:
import wandb

wandb.login()

wandb: Currently logged in as: srichaiperksiv (dads7202). Use `wandb login --relogin` to force relogin


True

In [30]:
tokenizer.decode(batch['input_ids'][0])

'Text generation from keyword: Super Bowl Advertising revenue in million U.S. dollars Super Bowl XLVI (2012) 245 Super Bowl XLIII (2009) 204 Super Bowl XLV (2011) 195 Super Bowl XLII (2008) 186 Super Bowl XL (2006) 163 Super Bowl XXXIX (2005) 158 Super Bowl XLI (2007) 152 Super Bowl XLIV (2010) 151 Super Bowl XXXVIII (2004) 150 Super Bowl XXXVI (2002) 134</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad

In [31]:
labels = batch['labels'][0]
tokenizer.decode([label for label in labels if label != -100])

'The statistic shows the most valuable Super Bowls based on total advertising revenue. Super Bowl XLVI ( 2012 ) is expected to be the most valuable Super Bowl with a total ad revenue of 245 million U.S. dollars. More Super Bowl statistics</s>'

In [32]:
model = CodeT5()

### Train (fit) model1

***** ถ้าใช้ k2t ให้ #logger=wandb_logger ด้วย **********

In [33]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor

import sys
import signal

save_directory = "/content/drive/MyDrive/DADS7202_final/model1-t5base-4000d-100e-256-v1-timeout"

def save_and_exit(signal, frame):
    model.model.save_pretrained(save_directory)
    sys.exit(0)

signal.signal(signal.SIGINT, save_and_exit)


wandb_logger = WandbLogger(name='codet5-finetune-code-summarization-ruby-shuffle', project='CodeT5')
# for early stopping, see https://pytorch-lightning.readthedocs.io/en/1.0.0/early_stopping.html?highlight=early%20stopping
early_stop_callback = EarlyStopping(
    monitor='validation_loss',
    patience=10,
    strict=False,
    verbose=False,
    mode='min'
)
lr_monitor = LearningRateMonitor(logging_interval='step')

trainer = Trainer(gpus=1, 
                  default_root_dir="/content/drive/MyDrive/CodeT5/Notebooks/Checkpoints", 
                  logger=wandb_logger, 
                  callbacks=[early_stop_callback, lr_monitor])
trainer.fit(model)

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:441: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:30

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

#### check layers and weight in model

In [60]:
# Access the state_dict attribute to get a dictionary of weights
weights = model.state_dict()

# Print the keys and shapes of the weights
#for key, value in weights.items():
    #print(key, value.shape)

In [ ]:
# Get the weights of the first layer
layer_weights = weights['model.decoder.final_layer_norm.weight']

# Print the shape of the weights
print(layer_weights)

************* ตั้ง path ที่จะ save model อันแรก ***********

In [34]:
save_directory = "/content/drive/MyDrive/DADS7202_final/model1-t5base-4000d-100e-256-v1"

In [35]:
# save in the current working directory, you can change this of course
model.model.save_pretrained(save_directory)

In [36]:
dataset2 = dataset_dict
print(dataset2['test'])

Dataset({
    features: ['key', 'text'],
    num_rows: 600
})


In [37]:
test_example = dataset2['test'][0]
print("Key:", test_example['key'])

Key: Religion Percentage No religious denomination 52% Roman Catholic 23% Dutch Reformed Church 6% Protestant Church in the Netherlands 6% Reformed Churches in the Netherlands 3% Other 10%


In [38]:
test_example = dataset2['test'][0]
print("Text:", test_example['text'])

Text: This statistic shows the population of the Netherlands by religion in 2018 . This statistic shows that in 2018 roughly half of the Dutch population did not identify with any religious denomination . Of the religious population , the largest number of people ( nearly 25 percent ) identified themselves as Roman Catholics . That is particularly interesting , as the dominating church in the Netherlands since the Reformation had been the Protestant church . According to the source , approximately 15 percent of the Dutch population considered themselves a member of the three main protestant churches , the Dutch Reformed Church , the Protestant Church in the Netherlands and the Reformed Churches in the Netherlands . The category 'other ' includes not only the smaller protestant churches , but also religions that are newer to the Netherlands , such as Buddhism , Hinduism and Islam . According to this statistic , Muslims made up five percent of the population in the Netherlands . 


In [36]:
#from transformers import T5ForConditionalGeneration
#model = T5ForConditionalGeneration.from_pretrained(save_directory)

**************** AutoModelForSeq2SeqLM ของ แต่ละ model ให้ดูตาม huggingface ******************

In [39]:
model = AutoModelForSeq2SeqLM.from_pretrained(save_directory)

### Generate data from own model

In [40]:
# prepare for the model
input_ids = tokenizer(test_example['key'], return_tensors='pt').input_ids
# generate
outputs = model.generate(input_ids,min_length=150, max_length=200)
print("Generated :", tokenizer.decode(outputs[0], skip_special_tokens=True))

Generated : This statistic shows the percentage of Dutch people who have no religious denomination in the Netherlands as of January 2019, by religion. In that year, 52 percent of Dutch people had no religious denomination at all. The Dutch Reformed Church was the second most popular religion in the Netherlands, with a 6% membership rate. The Dutch Reformed Church was the second most popular religion in the Netherlands, with a 6% membership rate. The Dutch Reformed Church was founded in the Netherlands in 1789. The Dutch Reformed Church is the most popular type of Christian denomination in the Netherlands. The Dutch Reformed Church is the most popular type of Christian denomination in the Netherlands, with a 63 percent membership rate. The Dutch Reformed Church is the second most popular type of Christian denomination in the Netherlands, with a 63 percent membership rate. The Dutch Reformed Church is the most popular type of Christian denomination in the


Create list of predict data and ref data

In [42]:
predict = []
refer = []
for i in range(600):
  test_example = dataset2['test'][i]
  input_ids = tokenizer(test_example['key'], return_tensors='pt').input_ids
  outputs = model.generate(input_ids,min_length=150, max_length=200)
  predict.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
  refer.append(test_example['text'])

Token indices sequence length is longer than the specified maximum sequence length for this model (548 > 512). Running this sequence through the model will result in indexing errors


In [43]:
print(len(predict))
print(len(refer))

600
600


In [47]:
'''## save list to txt
with open("predict.txt", "w") as output:
    output.write(str(predict))

with open("refer.txt", "w") as output:
    output.write(str(refer))'''

### Evaluation

In [44]:
pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 81 kB 7.3 MB/s 


In [45]:
import evaluate
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=predict, references=refer)
print(results)

{'bleu': 0.10291235965013247, 'precisions': [0.24569362950544846, 0.11643821172501054, 0.07376910016977929, 0.05315036309269543], 'brevity_penalty': 1.0, 'length_ratio': 1.7463860933211346, 'translation_length': 95440, 'reference_length': 54650}


In [48]:
pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [49]:
from rouge import Rouge 

hypothesis = "the #### transcript is a written version of each day 's cnn student news program use this transcript to he    lp students with reading comprehension and vocabulary use the weekly newsquiz to test your knowledge of storie s you     saw on cnn student news"

reference = "this page includes the show transcript use the transcript to help students with reading comprehension and     vocabulary at the bottom of the page , comment for a chance to be mentioned on cnn student news . you must be a teac    her or a student age # # or older to request a mention on the cnn student news roll call . the weekly newsquiz tests     students ' knowledge of even ts in the news"

rouge = Rouge()
scores = rouge.get_scores(predict, refer, avg = True)
print(scores)

{'rouge-1': {'r': 0.45283061464010066, 'p': 0.3963436360926737, 'f': 0.38653084354934125}, 'rouge-2': {'r': 0.25397911670966217, 'p': 0.16933723599851908, 'f': 0.1832125442271845}, 'rouge-l': {'r': 0.4377530315267607, 'p': 0.37975063066534603, 'f': 0.37214064732982644}}


## Create data from Ielt DATASET

*********** แก้ path ************

In [ ]:
List_key = []
for i in range(len(os.listdir('/content/drive/MyDrive/DADS7202_final/IELTS/table'))): 
  src = '/content/drive/MyDrive/DADS7202_final/IELTS/table/'+str(i)+'.png'
  img = cv2.imread(src)
  # Adding custom options
  custom_config = r'--psm 1'
  text = pytesseract.image_to_string(img, config=custom_config)
  #print(text)
  text_split = text.split('\n')
  #print(text_split)
  while '' in text_split:
    text_split.remove('')
  while ' ' in text_split:
    text_split.remove(' ')
  key = ' '.join(text_split[:-2])
  List_key.append(key)
len(List_key)

In [54]:
import pickle

with open('List_key_ielts.pkl', 'wb') as f:
    pickle.dump(List_key, f)

Load ielts key

In [51]:
import pickle

with open('List_key_ielts.pkl', 'rb') as f:
    List_key = pickle.load(f)

In [130]:
txt_key = []
for i in range(len(os.listdir('/content/drive/MyDrive/DADS7202_final/IELTS/captions'))): 
  f = open('/content/drive/MyDrive/DADS7202_final/IELTS/captions/'+str(i)+'.txt', 'r')
  content = f.read()
  content = re.sub('\s+',' ',content)
  txt_key.append(content)
len(txt_key)

201

In [58]:
import pickle

with open('txt_key_ielts.pkl', 'wb') as f:
    pickle.dump(txt_key, f)

Load ielts txt

In [52]:
import pickle

with open('txt_key_ielts.pkl', 'rb') as f:
    txt_key = pickle.load(f)

In [53]:
import pyarrow as pa

# Create a list of lists that represents the data
train_data = [List_key[:140] ,txt_key[:140]]

# Create a list of pyarrow.Array objects from the data
arrays_train = [pa.array(col) for col in train_data]

# Create a pyarrow Table object from the arrays
table1 = pa.Table.from_arrays(arrays_train, names=['key', 'text'])
#print(type(table1))

# Create a Dataset object from the table
dataset1 = Dataset(table1)
#print(type(dataset1))

# Create a list of lists that represents the data
val_data = [List_key[140:170] ,txt_key[140:170]]

# Create a list of pyarrow.Array objects from the data
arrays_val = [pa.array(col) for col in val_data]

# Create a pyarrow Table object from the arrays
table2 = pa.Table.from_arrays(arrays_val, names=['key', 'text'])
#print(type(table1))

# Create a Dataset object from the table
dataset2 = Dataset(table2)
print(type(dataset2))

# Create a list of lists that represents the data
test_data = [List_key[170:] ,txt_key[170:]]

# Create a list of pyarrow.Array objects from the data
arrays_test = [pa.array(col) for col in test_data]

# Create a pyarrow Table object from the arrays
table3 = pa.Table.from_arrays(arrays_test, names=['key', 'text'])
#print(type(table1))

# Create a Dataset object from the table
dataset3 = Dataset(table3)
print(type(dataset3))

# Create a dictionary that maps keys to the Dataset objects
ielts_dataset = {
    'train': dataset1,
    'validation': dataset2,
    'test': dataset3
}

# Access a Dataset object using its key
print(ielts_dataset)  # Output: Dataset object with train_data
print(type(ielts_dataset))
#print(dataset['validation'])  # Output: Dataset object with validation_data
#print(dataset['test'])  # Output: Dataset object with test_data

<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>
{'train': Dataset({
    features: ['key', 'text'],
    num_rows: 140
}), 'validation': Dataset({
    features: ['key', 'text'],
    num_rows: 30
}), 'test': Dataset({
    features: ['key', 'text'],
    num_rows: 31
})}
<class 'dict'>


In [54]:
print(ielts_dataset['train'][0])

{'key': 'Holiday leisure activities Number of hours |At the beach Reading books Chinese tourists 3 8 American tourists 4 5. Turkish tourists 5 4', 'text': 'The bar chart compares the amount of time each day that people from four different countries spent doing activities while on holiday in Greece in August 2019. The activities were going to the beach, visiting different attractions, and reading. Overall, the tourists spent the most time reading books while they spent the least time visiting places. The Chinese tourists spent the longest reading books with a total of eight hours per day. By contrast, the Brazilians spent the longest at the beach with a total of six hours. Both the American and the Turkish tourists spent the same number of hours every day visiting places with a total of three hours each. Similarly, the Turkish and Brazilian groups spent a total of four hours each reading books. The Chinese tourists spent twice the amount of time reading books as the Turkish and Brazilia

In [55]:
from datasets import DatasetDict

dataset_dict = DatasetDict(ielts_dataset)
print(type(dataset_dict))
print(dataset_dict)

<class 'datasets.dataset_dict.DatasetDict'>
DatasetDict({
    train: Dataset({
        features: ['key', 'text'],
        num_rows: 140
    })
    validation: Dataset({
        features: ['key', 'text'],
        num_rows: 30
    })
    test: Dataset({
        features: ['key', 'text'],
        num_rows: 31
    })
})


**Train ielts data**

In [56]:
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForSeq2SeqLM

*********** แก้ tokenizer ตาม model เรา **********

In [57]:
tokenizer = AutoTokenizer.from_pretrained("t5-base")

prefix = "Text generation from keyword: "
max_input_length = 512
max_target_length = 256

def preprocess_examples(examples):
  # encode the code-docstring pairs
  codes = examples['key']
  docstrings = examples['text']
  
  inputs = [prefix + code for code in codes]
  model_inputs = tokenizer(inputs, max_length=max_input_length, padding="max_length", truncation=True)

  # encode the summaries
  labels = tokenizer(docstrings, max_length=max_target_length, padding="max_length", truncation=True).input_ids

  # important: we need to replace the index of the padding tokens by -100
  # such that they are not taken into account by the CrossEntropyLoss
  labels_with_ignore_index = []
  for labels_example in labels:
    labels_example = [label if label != 0 else -100 for label in labels_example]
    labels_with_ignore_index.append(labels_example)
  
  model_inputs["labels"] = labels_with_ignore_index

  return model_inputs

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [58]:
dataset = dataset_dict.map(preprocess_examples, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [59]:
from torch.utils.data import DataLoader

dataset.set_format(type="torch", columns=['input_ids', 'attention_mask', 'labels'])
train_dataloader = DataLoader(dataset['train'], shuffle=True, batch_size=8)
valid_dataloader = DataLoader(dataset['validation'], batch_size=4)
test_dataloader = DataLoader(dataset['test'], batch_size=4)

In [60]:
batch = next(iter(train_dataloader))
print(batch.keys())

dict_keys(['input_ids', 'attention_mask', 'labels'])


In [61]:
!pip install -q pytorch-lightning wandb

### Create CodeT5 (model2)

*********** แก้ AutoModelForSeq2SeqLM ตามแต่ละโมเดลตาม huggingface ********

********* แก้ epoch ***********

In [62]:
from transformers import T5ForConditionalGeneration, AdamW, get_linear_schedule_with_warmup
import pytorch_lightning as pl

class CodeT5(pl.LightningModule):
    def __init__(self, lr=5e-5, num_train_epochs=100, warmup_steps=100):  #edit here
        super().__init__()
        self.model = AutoModelForSeq2SeqLM.from_pretrained(save_directory)  #edit here
        self.save_hyperparameters()

    def forward(self, input_ids, attention_mask, labels=None):     
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        return outputs
    
    def common_step(self, batch, batch_idx):
        outputs = self(**batch)
        loss = outputs.loss

        return loss
      
    def training_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     
        # logs metrics for each training_step,
        # and the average across the epoch
        self.log("training_loss", loss)

        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     
        self.log("validation_loss", loss, on_epoch=True)

        return loss

    def test_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     

        return loss

    def configure_optimizers(self):
        # create optimizer
        optimizer = AdamW(self.parameters(), lr=self.hparams.lr)
        # create learning rate scheduler
        num_train_optimization_steps = self.hparams.num_train_epochs * len(train_dataloader)
        lr_scheduler = {'scheduler': get_linear_schedule_with_warmup(optimizer,
                                                    num_warmup_steps=self.hparams.warmup_steps,
                                                    num_training_steps=num_train_optimization_steps),
                        'name': 'learning_rate',
                        'interval':'step',
                        'frequency': 1}
        
        return {"optimizer": optimizer, "lr_scheduler": lr_scheduler}

    def train_dataloader(self):
        return train_dataloader

    def val_dataloader(self):
        return valid_dataloader

    def test_dataloader(self):
        return test_dataloader

In [ ]:
import wandb

wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [63]:
tokenizer.decode(batch['input_ids'][0])

'Text generation from keyword: Time spent on smartphones and tablets, by category Percentage of time spent Smartphones| Tablet Games 35% 57% Social Network 29% 15% Utilities 20% 3% Music and videos 8% 13% News 3% 4%</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><

In [64]:
labels = batch['labels'][0]
tokenizer.decode([label for label in labels if label != -100])

'Overall, the two pie charts show that smartphones and tablets are used for the same purposes but to very different extents. The first pie chart shows how people spend their time on smartphones while the second pie chart illustrates how time is spent on tablets. For both types of device, the top use is for games, but the figures differ greatly. 57% of the time spent on a tablet is given to playing games, while only 35% of the time spent on a smartphone is used for this. In contrast, smartphone users spend 29% of their time on their gadget accessing social networking sites compared with just 15% of tablet time spent on the same activity. The third most popular use of the tablet is for consuming entertainment, with users spending 13% of their tablet time watching videos and listening to music. Smartphone users, however, dedicate only 8% of their smartphone time to such entertainment, preferring instead to spend 20% of their time on their phone accessing utilities. These can include maps,

Create new model to train again

In [65]:
model = CodeT5()

### Train (fit) model2

In [66]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor

wandb_logger = WandbLogger(name='codet5-finetune-code-summarization-ruby-shuffle', project='CodeT5')
# for early stopping, see https://pytorch-lightning.readthedocs.io/en/1.0.0/early_stopping.html?highlight=early%20stopping
early_stop_callback = EarlyStopping(
    monitor='validation_loss',
    patience=10,
    strict=False,
    verbose=False,
    mode='min'
)
lr_monitor = LearningRateMonitor(logging_interval='step')

trainer = Trainer(gpus=1, 
                  default_root_dir="/content/drive/MyDrive/CodeT5/Notebooks/Checkpoints", 
                  #logger=wandb_logger, 
                  callbacks=[early_stop_callback, lr_monitor])
trainer.fit(model)

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/loggers/wandb.py:396: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:441: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/loops/utilities.py:94: 

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/trainer.py:1595: PossibleUserWarning: The number of training batches (18) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower val

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

******* set new directory ที่จะsaveใน model อันใหม่หลัง train กับ ielts แล้ว ********

In [67]:
save_directory = "/content/drive/MyDrive/DADS7202_final/model2-t5base-4000d-100e-256-v1"

In [68]:
 # save in the current working directory, you can change this of course
model.model.save_pretrained(save_directory)

In [69]:
dataset2 = dataset_dict
print(dataset2['test'])

Dataset({
    features: ['key', 'text'],
    num_rows: 31
})


In [70]:
test_example = dataset2['test'][0]
print("Key:", test_example['key'])

Key: yearly income year Amandline Bakesry Mari Bakeshop Bolo Cakery 2001 20,000.00 80,000.00 58,000.00 2002 25,000.00 95,000.00 58,000.00 2003 21,000.00 90,000.00 58,000.00 2004 39,000.00 80,000.00 58,000.00 2005 39,000.00 92,000.00 58,000.00 2006 38,000.00 61,000.00 60,000.00 2007 41,500.00 59,000.00 66,000.00 2008 62,000.00 60,000.00 97,000.00 2009 62,500.00 41,000.00 105,000.00


In [71]:
test_example = dataset2['test'][0]
print("Text:", test_example['text'])

Text: The graph shows information about the amount of money which was earned by three bakeries in Calgary, over a ten-year period between 2000 and 2010. Overall, what stands out from the graph is that there were considerable upward trends in the income of both Bolo Cakery and Amandine Bakery, while the earnings of Mari Bakeshop saw a considerable fall over the period in question. Another interesting point is that Mari Bakeshop was the most popular bakery in 2000, but in 2010, Amandine Bakery earned more money than the others. Looking at the details, as regards Bolo Cakery, income started at $20,000 in 2000, then there was a fluctuation over the next three years, at which point it levelled off at just under $40,000 until 2006. Then the figure went up significantly, finishing at around $65,000 in 2010. If we look at Amandine Bakery, the trend was similar. Having remained stable at approximately $55,000 in the first half of the decade, income then rose sharply, reaching nearly $100,000 in

******** แก้ AutoModelForSeq2SeqLM ตาม huggingface

### Generate data from ielts model

In [72]:
model = AutoModelForSeq2SeqLM.from_pretrained(save_directory)

In [73]:
# prepare for the model
input_ids = tokenizer(test_example['key'], return_tensors='pt').input_ids
# generate
outputs = model.generate(input_ids,min_length=150, max_length=200)
print("Generated :", tokenizer.decode(outputs[0], skip_special_tokens=True))

Generated : The table shows the annual income of the bakers in the years 2001, 2000 and 2009. The annual income of the bakers increased from 20,000.00 to 80,000.00. In 2001, the income of the bakers increased from 20,000.00 to 80,000.00. In 2002, the income of the bakers increased from 25,000.00 to 95,000.00. In 2003, the annual income of the bakers increased from 25,000.00 to 95,000.00. In 2006, the annual income of the bakers increased from 39,000.00 to 62,000.00. In 2009, the annual income of the bakers increased from 60,000.00 to 60,000.00. In 2001, the annual income of the bakers increased from 20,000.00 to 60,000.00. In 2003, the annual income of the bakers increased from 25,000.00 to 25,000.00. In 2003, the annual income of the bakers increased from 25,000.00 to


### Evaluation

In [74]:
predict = []
refer = []
for i in range(31):
  test_example = dataset2['test'][i]
  input_ids = tokenizer(test_example['key'], return_tensors='pt').input_ids
  outputs = model.generate(input_ids,min_length=150, max_length=200)
  predict.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
  refer.append(test_example['text'])

In [75]:
print(len(predict))
print(len(refer))

31
31


In [77]:
i = 11
print(predict[i])
print(refer[i])

The proportion of tourists who visited Brighton attractions in 1980 was much higher than the proportion of tourists who visited Brighton attractions. In 1980, 22% of tourists went to Brighton attractions, but this figure rose to 23%. By 2000, this figure had fallen to 38%. By 2000, this figure had fallen to 35%. By 2005, this figure had fallen to 27%. By 2005, this figure had fallen to 27%. By 2005, this figure had fallen to 27%. By 2005, this figure had fallen to 27%. By 2005, this figure had fallen to 27%, and it was only 11% of tourists. In 1980, the Art Gallery Pavilion Pier Festival had the highest percentage of tourists who visited Brighton attractions. By 2005, this figure had fallen to 27%. In 1985, this figure had fallen to 15%, and it had fallen to 15%. In 2005, this figure had fallen to 15%. In 1990, this figure had fallen to 15%, and it
The line graph shows the percentage of tourists to England who visited certain Brighton attractions between 1980 and 2010. We can see that 

In [ ]:
'''## save list to txt
with open("predict.txt", "w") as output:
    output.write(str(predict))

with open("refer.txt", "w") as output:
    output.write(str(refer))'''

In [ ]:
#pip install evaluate

In [78]:
import evaluate
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=predict, references=refer)
print(results)

{'bleu': 0.051760140049699706, 'precisions': [0.34762163139808283, 0.08439432520916697, 0.025242363270532284, 0.009933774834437087], 'brevity_penalty': 0.9938694747230097, 'length_ratio': 0.9938881898256337, 'translation_length': 5529, 'reference_length': 5563}


In [74]:
#pip install rouge

In [79]:
from rouge import Rouge 

hypothesis = "the #### transcript is a written version of each day 's cnn student news program use this transcript to he    lp students with reading comprehension and vocabulary use the weekly newsquiz to test your knowledge of storie s you     saw on cnn student news"

reference = "this page includes the show transcript use the transcript to help students with reading comprehension and     vocabulary at the bottom of the page , comment for a chance to be mentioned on cnn student news . you must be a teac    her or a student age # # or older to request a mention on the cnn student news roll call . the weekly newsquiz tests     students ' knowledge of even ts in the news"

rouge = Rouge()
scores = rouge.get_scores(predict, refer, avg = True)
print(scores)

{'rouge-1': {'r': 0.20651920015516084, 'p': 0.49978521645883384, 'f': 0.28753533342782955}, 'rouge-2': {'r': 0.05396891066536331, 'p': 0.1317981258729265, 'f': 0.0750769605519461}, 'rouge-l': {'r': 0.1957815132294777, 'p': 0.472509972606696, 'f': 0.2723959359871263}}
